In [ ]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
import moola
from SimpleBrainMesh import generate_doughnut_mesh
from BiotSolver import solve_biot
from fenics_adjoint import *
from pyadjoint import ipopt
set_log_level(30)


T = 5            # final time
num_steps = 50     # number of time steps
dt = T / num_steps # time step size


N = 20 # resolution
brain_radius = 0.1  # brain circle Radius
ventricle_radius = brain_radius/3
aqueduct_width = brain_radius/5

a_0 = 0.6
a_1 = 1.5
f_0 = 1.0/(60/15)
f_1 = 1.0/(60/70)
c = 7 

kappa = 1.5*1e-11    # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity
E = 1500.0          # Young modulus
nu = 0.49999        # Poisson ratio

c = 2*1e-4  # storage coefficent
alpha = 0.5 # Biot-Willis coefficient (change of the bulk volume due to pore pressure change)

# Lame coefficients
lmbda = nu*E/((1.0-2.0*nu)*(1.0+nu)) 
mu = E/(2.0*(1.0+nu))

mmHg2Pa = 133.32
material_parameter = dict()
material_parameter["c"] = c
material_parameter["K"] = K 
material_parameter["lambda"] = lmbda
material_parameter["mu"] = mu
material_parameter["alpha"] = alpha

# create mesh

mesh = Mesh(generate_doughnut_mesh(brain_radius, ventricle_radius,
                                   aqueduct_width, N))

ventricle = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] < R*R*0.99)",
                              R = brain_radius)
skull = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] >= R*R*0.99 )",
                          R = brain_radius)

p_obs = Expression("""c*mmHg2Pa*(a_0 * sin(2 *pi * f_0*t) + a_1 *sin(2*pi*f_1*t))
                   *( sqrt( x[0]*x[0] + x[1]*x[1] ) - R_ventricle ) """,
                    a_0=a_0, a_1=a_0, f_0=f_0, f_1=f_1, t = 0,c=c,
                    mmHg2Pa=mmHg2Pa, R_ventricle=ventricle_radius,
                    degree = 2)

p_D_skull = p_obs
p_D_ventricle = p_obs

u_D_skull = Constant([0.0, 0.0])

probe_points = [
                Point(ventricle_radius + 0.2*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.4*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.6*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.8*(brain_radius - ventricle_radius), 0)
                ]

x_coords = np.linspace(ventricle_radius, brain_radius, 1000)
slice_points = [Point(x, 0.0) for x in x_coords]

In [ ]:
results = solve_biot(mesh, material_parameter, T, num_steps, 
                     skull, ventricle,
                     p_D_skull=p_D_skull,
                     p_D_ventricle=p_D_ventricle,
                     u_D_skull=u_D_skull,
                     probe_points=probe_points,
                     slice_points=slice_points)

In [ ]:
times = [1,2,3,4,5]

idx = []
for i in range(num_steps):
    if np.isclose(i*dt, times).any():
        idx.append(i)
cross_section = results["p_fluid_slice"]
plt.figure(figsize=(10,10))
for i in idx:
    plt.plot(x_coords, cross_section[i,:], label="t = {}".format(i*dt))
plt.legend()
plt.grid()
plt.xlabel("x in m")
plt.ylabel("p in Pa")


In [ ]:
p_data = results["probe_point_p_fluid"]
plt.figure(figsize=(10,10))
num_points = len(probe_points)
for i in range(num_points):
    plt.plot(np.linspace(0,T, num_steps), 
             p_data[:,i], 
             label="$P_{}$".format(i)
            )
plt.xlabel("t in s")
plt.ylabel("p in Pa")
plt.grid()
plt.legend()


In [ ]:
plt.figure(figsize=(8,8))
cont = plot(mesh)
for i, point in enumerate(probe_points):
    plt.scatter(point.array()[0], point.array()[1],
                marker=".", s= 300)
    plt.annotate( "$P_{}$".format(i), point.array()[0:2], size=20)